# OIH Data Science Notebook:  Geocoverage inspection on combined parquet

This notebook demonstrates query approach for the pre-processed resources from the OIH Graph

Notes:

Need to download the model with _python -m spacy download en_core_web_lg_

you can also downloiad with

## Note

```
pip install -q
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz

pip install -q -U spacy[cuda113]

```

## Model loading

In [31]:
!python -m spacy download en_core_web_lg --quiet
# !python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


## Imports

In [20]:
import duckdb
import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning

import spacy
# import scispacy
#import en_core_sci_sm
from spacy import displacy
import getpass
import pandas as pd
import urllib.request, json
import en_core_web_sm  # intersting need for import here...
import numpy as np
from multiprocessing import Pool
# import dask, boto3
# import s3fs

## Load Pre-processed OIH Graph

In [5]:
## load the combined graph
url = "http://ossapi.oceaninfohub.org/public/combined.parquet"
duckdb.install_extension("httpfs")

# Instantiate the DuckDB connection
con = duckdb.connect()
con.execute("CREATE TABLE my_table AS SELECT * FROM read_parquet('{}')".format(url))  # load from url
# con.execute("CREATE TABLE my_table AS SELECT * FROM read_parquet('/content/drive/MyDrive/Data/combined.parquet')") # load from local parquet
# con.execute("CREATE TABLE data AS SELECT row_number() OVER () AS idx, * FROM './inputs/combined.parquet';")

## Spacy test

In [30]:
from spacy import displacy
import spacy
import en_core_web_sm  # intersting need for import here...

text = "Estonian Marine Institute is one of many Tartu University&rsquo;s contemporary scientific institutions. The main purpose of Estonian Marine Institute is marine research and promotion of the given sphere in Estonia and the Baltic region. This institute is one of the biggest organizations in Estonia carrying out marine exploration contributing research in several marine study fields. Our research ranges from water physics to biology, from microscopic scale to ecosystems having unique expert opinion and qualification in most research fields. Estonian Marine Institute is an educational basis for the marine biology-oriented postgraduate students and is actively improving the advanced and extensive higher marine education in Estonia. The active progress of marine sciences in Estonia in the last decades has enhanced the awarness of Estonian science around the Baltic Sea countries and also in Europe. Estonian Marine Institute has very international nature and thus the research is conducted in high level and in close co-operation with other specialist around the world. The success of the Estonian marine scientists at the international level refers to their high-rated scientific publications and increase in the number of international projects. On the other hand a significant part of the research is aimed at finding solutions to the local scientific problems important for Estonia&rsquo;s well-being. We hope our website provides you easy access to all of the information you seek about the Estonian Marine Institute and useful links to other related themes."

nlp = spacy.load("en_core_web_lg")
# nlp = spacy.load("en_core_web_trf")
# doc = nlp(''.join(kwtest)) # nlp(text)
doc = nlp(text) # nlp(text)

displacy_image = displacy.render(doc, jupyter = True, style = 'ent')

## DuckDB test querries 

In [45]:
# Now you can execute SQL queries on the Parquet file as if it was a regular table
# r = con.execute("SELECT DISTINCT provder FROM my_table").fetchdf()
# r = con.execute(" SELECT DISTINCT provder, type, ANY_VALUE(s),  COUNT(*) AS count FROM my_table GROUP BY provder, type  order by count desc").fetchdf()
# r = con.execute(" SELECT DISTINCT provder, type, COUNT(*) AS count FROM my_table GROUP BY provder, type").fetchdf()
# r = con.execute('SELECT "desc" FROM data LIMIT 1').fetchdf()
# r = con.execute('SELECT "desc" FROM data WHERE idx = {};'.format("22222")).fetchdf()
# r = con.execute('SELECT CONCAT(name, "desc") AS full_name FROM data;').fetchdf()
# print(r.iloc[1010,0])

r = con.execute('SELECT COUNT(*) FROM data').fetchdf()
len = r.iloc[0,0]
print(len)

685406


## Simple Loop with DuckDB calls

In [ ]:

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

# nlp = en_core_web_sm.load()
# nlpdf = pd.DataFrame(columns=['name', 'label', 'text', 'url'])
nlpdf = pd.DataFrame(columns=['label', 'text'])


# for i in range(len(oihdf)):
# for i in tqdm.tqdm(range(len(oihdf))):
for i in  tqdm.tqdm(range(len)):
    r = con.execute('SELECT "desc" FROM data WHERE idx = {};'.format(str(i+1))).fetchdf()
    text = str(r.iloc[0,0])
    # text = ' '.join(oihdf.at[i,'desc'].astype(str))
    if not isinstance(text, str) or text != "nan":
      # print(text)
      doc = nlp(text)
      for entity in doc.ents:
          nlpdf = nlpdf.append({'label': entity.label_, 'text':entity.text}, ignore_index=True)


## Multiprocessing Pool over DuckDB

In [21]:
%%time

import multiprocessing

# spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")

r = con.execute('SELECT "desc" FROM data;').fetchdf()

# nlp = en_core_web_sm.load()
# nlpdf = pd.DataFrame(columns=['name', 'label', 'text', 'url'])

def getents(number):
  nlpdf = pd.DataFrame(columns=['label', 'text'])  # ['name', 'label', 'text', 'url'])
  r = con.execute('SELECT "desc" FROM data WHERE idx = {};'.format(str(number))).fetchdf()
  text = str(r.iloc[0,0])
  # text = ' '.join(oihdf.at[i,'desc'].astype(str))
  if not isinstance(text, str) or text != "nan":
    # print(text)
    doc = nlp(text)
    for entity in doc.ents:
        nlpdf = nlpdf.append({'label': entity.label_, 'text':entity.text}, ignore_index=True)

  return nlpdf


# Create a process pool with the desired number of processes
pool = multiprocessing.Pool(processes=10)

# Generate a range of numbers from 1 to 10,000
numbers = range(1, len)
# numbers = range(1, 1000)

# Map the calculation function to the numbers in the pool
results = pool.map(getents, numbers)

# Close the process pool.
pool.close()
pool.join()


CPU times: user 1min 59s, sys: 5.96 s, total: 2min 5s
Wall time: 52min 11s


In [22]:
%%time 
collecteddfs = pd.concat(results)


CPU times: user 25.9 s, sys: 220 ms, total: 26.1 s
Wall time: 26.1 s


In [23]:
collecteddfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9884367 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   label   object
 1   text    object
dtypes: object(2)
memory usage: 226.2+ MB


In [24]:
filtered_df = collecteddfs[collecteddfs['label'] == 'GPE']

# Count the occurrences of each unique term in the "text" column
counts = filtered_df['text'].value_counts()

# Convert the counts Series to a DataFrame
new_df = counts.reset_index()

# Rename the columns
new_df.columns = ['place', 'count']



In [27]:
new_df.head(5)

,place,count
0,Argentina,68142
1,Florida,18849
2,Iran,13233
3,Colpophyllia,9308
4,British Columbia,8545


In [26]:
new_df.to_csv('descCounts.csv', index=False)
